# KineticsデータセットでECO用のDataLoaderを作成する

## 目標
1.	Kinetics動画データセットをダウンロードできるようになる
2.	動画データをフレームごとの画像データに変換できるようになる
3.	ECOで使用するためのDataLoaderを実装できるようになる

## Library

In [1]:
import os
from PIL import Image
import csv
import numpy as np

import torch
import torch.utils.data
from torch import nn

import torchvision

## 動画を画像データにしたフォルダへのファイルパスのリスト作成

9-4_2_convert_mp4_to_jpeg.ipynbで画像にした。

In [2]:
def make_datapath_list(root_path):
    '''
    動画を画像データにしたフォルダへのファイルパスリストを作成
    Params
    ---------
    root_path : str 
        データフォルダへのrootパス
        
    Returns
    ---------
    ret : video_list
        動画を画像にしたフォルダへのファイルパスリスト
    '''
    
    # 動画を画像データにしたフォルダへのファイルパスリスト
    video_list = []
    
     # root_pathにあるディレクトリ（クラスの種類）のパスを取得
    class_list = os.listdir(path=root_path)
    
    # 各クラスの動画フィアルを画像化したフォルダへのパスを取得
    for class_list_i in class_list:  # クラスごとのループ
        
        # クラスフォルダへのパスを取得
        class_path = os.path.join(root_path, class_list_i)
        
        # 各クラスのフォルダ内の画像ファイルを取得するループ
        for file_name in os.listdir(class_path):
            
            # ファイル名と拡張子に分割
            name, ext = os.path.splitext(file_name)
            
            # 拡張子が'.mp4'でないファイルは無視する
            if ext == '.mp4':
                continue
                
            # 動画ファイルを画像に分割して保存したフォルダのパスを取得
            video_img_directory_path = os.path.join(class_path, name)
            
            # video_listに追加
            video_list.append(video_img_directory_path)
            
    return video_list

# 動作確認
root_path = './data/kinetics_videos/'
video_list = make_datapath_list(root_path)
# 1番目と２番めの動画の画像が格納されているディレクトリ
print(video_list[0])
print(video_list[1])
            

## 動画の前処理クラスの作成

In [9]:
class VideoTransform():
    '''
    動画を画像にした画像ファイルの前処理クラス
    学習時と推論時で異なる動作をさせる →　DataAugmentation省略しているので同じではないか？
    動画を画像に分割しているため、分割された画像たちをまとめて前処理する点に注意
    '''
    
    def __init__(self, resize, crop_size, mean, std):
        self.data_transform = {
            'train': torchvision.transforms.Compose([
                # DataAugmentationは今回は省略
                GroupResize(int(resize)),              # 画像をまとめてresize
                GroupCenterCrop(crop_size),      # 画像をまとめてCenterCrop
                GroupToTensor(),                          
                GroupImgNormalize(mean, std),  # 標準化
                Stack()                                            # 複数画像をframes次元で結合させる
            ]),
            'val': torchvision.transforms.Compose([
                GroupResize(int(resize)),              # 画像をまとめてresize
                GroupCenterCrop(crop_size),      # 画像をまとめてCenterCrop
                GroupToTensor(),                          
                GroupImgNormalize(mean, std),  # 標準化
                Stack()                                            # 複数画像をframes次元で結合させる
            ])
        }
        
    def __call_(self, img_group, phase):
        '''
        Params
        ---------
        phase : 'train' or 'val'
            前処理のモードを指定
        '''
        
        return self.data_transform[phase](img_group)

In [14]:
# 前処理クラスで使用するクラスの定義

class GroupResize():
    '''
    画像をまとめてResizeする
    画像の短い方の辺の長さがresizeに変換される
    アスペクト比は保たれる
    '''
    
    def __init__(self, resize, interpolation=Image.BILINEAR):
        '''リスケールする処理を用意（torchvisionから）'''
        self.rescaler = torchvision.transforms.Resize(resize, interpolation)
        
    def __call__(self, img_group):
        '''リスケールをimg_group(リスト)内の各imgに実施'''
        return [self.rescaler(img) for img in img_group]
    
    
class GroupCenterCrop():
    '''
    画像をまとめてCenterCropする
    (crop_size, crop_size)の画像を切り出す
    '''
    def __init__(self, crop_size):
        '''center cropする処理を用意（torchvisionから）'''
        self.ccrop = torchvision.transforms.CenterCrop(crop_size)
        
    def __call__(self, img_group):
        '''center cropをimg_group（リスト）内の各imgに実施'''
        return [self.ccrop(img) for img in img_group]
    
class GroupToTensor():
    '''画像をまとめてテンソル化するクラス'''
    
    def __init__(self):
        '''テンソル化する処理を用意する'''
        self.to_tensor = torchvision.transforms.ToTensor()
        
    def __call__(self, img_group):
        '''テンソル化をimg_group（リスト）内の各imgに実施
        0-1ではなく、0-255で扱うため　 *255
        0-255で扱うのは学習済みデータの形式に合わせるため
        '''
        
        # 画像データ自体がテンソルになっていればそれをまとめているのがテンソルでなくリストでもいいのか？
        # いや、最後のStack（）で
        return [self.to_tensor(img)*255 for img in img_group]
    

class GroupImgNormalize():
    '''
   画像をまとめて標準化するクラス
   ''' 
    def __init__(self, mean, std):
        '''標準化する処理を用意（torchvisionから）'''
        self.normalize = torchvision.transforms.Normalize(mean, std)
    
    def __call__(self, img_group):
        '''標準化をimg_group（リスト）内の各imgに実施'''
        return [self.normalize(img) for img in img_group]
    

class Stack():
    '''画像を１つのテンソルにまとめるクラス'''
    
    def __call__(self, img_group):
        '''img_groupはtorch.Size([3, 224, 224])を要素とするリスト'''
        
        ret = torch.cat([(x.flip(dims=[0])).unsqueeze(dim=0) for x in img_group], dim=0)
        # x.flip(dims=[0])で色チャネルを変更　RGB →　BGR
        # unsqueeze(dim=0)で各画像にframes用の次元を追加
        # frames次元で結合
        
        return ret

## Datasetの作成

In [ ]:
# kinetics-400のラベル名をIDに変換する辞書と逆にIDをラベル名に変換する辞書を用意

def get_label_dictionary(label_dictionary_path='./video_download/kinetics_400_label_dictionary.csv'):
    label_id_dict = {}
    id_label_dict = {}
    
    with open(label_dictionary_path, encoding='utf-8_sig') as f:
        
        # 読み込み
        reader = csv.DictReader(f, delimiter=',', quotechar="")
        
        # 1行ずつ読み込み、辞書型変数に追加
        for row in reader:
            # ラベル名をIDに変換
            label_id_dict.setdefault(
                row['class_label'], int(row['label_id'])-1)
            # IDをラベルに変換
            id_label_dict.setdeafault(
                int(row['label_id'])-1, row['class_label'])
            
    return label_id_dict, id_label_dict

# 確認
label_dictionary_path = './video_download/kinetics_400_label_dictionary.csv'
label_id_dict, id_label_dict = get_label_dictionary(label_dictionary_path)
label_id_dict

In [ ]:
class VideoDataset(torch.utils.data.Dataset):
    '''
    動画のDataset
    '''
    
    def __init__(self, video_list, )

## DataLoaderの作成